In [1]:
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm

# Dataset

In [2]:
train_data = pd.read_csv("../data/train_data.csv",dtype=np.uint8)
test_data = pd.read_csv("../data/test_data.csv",dtype=np.uint8)
train_labels = train_data["target"]
train_images = train_data.drop("target",axis = 1).values.reshape(-1,28,28,1)
test_labels = test_data["target"]
test_images = test_data.drop("target",axis = 1).values.reshape(-1,28,28,1)

# Hyperparameter


In [3]:
ksize = [3, 5, 7]   # gabor尺度
sigma = np.arange(2, len(ksize) + 2)
lambd = np.arange(3, len(ksize) + 3)   # gobar波长
theta = np.arange(0, np.pi, np.pi / 8) # gobar方向

# gabor function

$F(x,y) = exp(-\frac{x^2_0+\gamma^2y^2_0}{2\sigma^2})\times cos(\frac{2\pi}{\lambda}x_0)$

$x_0 = x cos(\theta)+y sin(\theta)$

$y_0 = -x sin(\theta)+y cos(\theta)$

In [4]:
def build_filters():
    filters = []
    for i in range(len(ksize)): 
        for j in range(len(theta)):
            kern = cv2.getGaborKernel((ksize[i], ksize[i]), sigma[i], theta[j], lambd[i], 0.5, 0, ktype=cv2.CV_32F)
            filters.append(kern)
    return filters

def gabor_process(img,filters): 
    process_imgs = []
    for kern in filters: 
        fimg = cv2.filter2D(img,cv2.CV_8UC1,kern) 
        process_imgs.append(fimg)
    return process_imgs

def pool(imgs):
    pool_imgs = []
    for img in imgs:
        row_n = img.shape[0]
        col_n = img.shape[1]
        pool_img = np.zeros((int(row_n/2),int(col_n/2)))
        for i in range(0,row_n,2):
            for j in range(0,col_n,2):
                pool_img[int(i/2),int(j/2)] = np.max(img[i:i+2,j:j+2])
        pool_imgs.append(pool_img)
    return pool_imgs
def flatten(imgs):
    flatten_imgs = []
    for img in imgs:
        flatten_imgs.extend(img.flatten())
    return np.array(flatten_imgs)

In [5]:
filters = build_filters()
gabor_process_train_imgs = []
for train_image in tqdm(train_images):
    gabor_process_imgs = gabor_process(train_image,filters)
    pool_process_imgs = pool(gabor_process_imgs)
    flatten_imgs = flatten(pool_process_imgs)/255
    gabor_process_train_imgs.append(flatten_imgs)

# gabor_process_train_imgs = pd.DataFrame(gabor_process_train_imgs)
# gabor_process_train_imgs["target"] = train_labels

 18%|█▊        | 10552/60000 [03:12<15:59, 51.56it/s]

KeyboardInterrupt: 

 18%|█▊        | 10552/60000 [03:30<15:59, 51.56it/s]

In [8]:
print(gabor_process_train_imgs[0])

[0. 0. 0. ... 0. 0. 0.]


gabor_process_test_imgs = []
for test_image in tqdm(test_images):
    gabor_process_imgs = gabor_process(test_image,filters)
    pool_process_imgs = pool(gabor_process_imgs)
    flatten_imgs = flatten(pool_process_imgs)/255
    gabor_process_test_imgs.append(flatten_imgs)